
# Bread Slicing Tutorial

In this tutorial, we will explore how to simulate the process of slicing a virtual loaf of bread. We'll calculate the positions and orientations of each slice and visualize them in relation to the bread's frame.

## Requirements:
- `pycram` library for bullet world simulation and reasoning.
- ROS message `PoseStamped` from `geometry_msgs` for pose information.



## Initial Setup

Let's start by importing the necessary libraries and setting up our virtual world.


In [ ]:
import pycram
from pycram.bullet_world import BulletWorld, Object
import pycram.bullet_world_reasoning as btr
import tf
from pycram.designators.motion_designator import MotionDesignatorDescription, MoveArmJointsMotion
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.language import macros, par
from pycram.designators.location_designator import *
from pycram.designators.action_designator import *
from pycram.enums import Arms
from pycram.designators.object_designator import *
from pycram.designators.object_designator import BelieveObject

world = BulletWorld()
world.set_gravity([0, 0, -9.8])
#plane = Object("floor", "environment", "plane.urdf", world=world)
robot = Object("pr2", "robot", "../../resources/" + robot_description.name + ".urdf")
robot_desig = ObjectDesignatorDescription(names=["pr2"]).resolve()


kitchen = Object("kitchen", "environment", "kitchen.urdf")



robot.set_joint_state(robot_description.torso_joint, 0.24)
kitchen_desig = ObjectDesignatorDescription(names=["kitchen"])

kitchen.set_color([0.2, 0, 0.4, 0.6])



## Setting up the World and Robot

We will define our virtual world with gravity and place a robot in it. Additionally, we'll load a kitchen environment to make our simulation more realistic.


In [ ]:
spawning_poses = {
    #'bigknife': Pose([-0.95, 1.2, 1.3], [1, -1, 1, -1]),
    'bigknife': Pose([-0.35, 1.2, 1.2], [0, 0, 0, 1]),
    'bread': Pose([-0.85, 0.9, 0.95],[0, 0, -1, 1])
}
bigknife = Object("bigknife", "bigkinfe", "big-knife.stl", spawning_poses["bigknife"])
bread = Object("bread", "bread", "bread.stl", spawning_poses["bread"])
bigknife_BO = BelieveObject(names=["bigknife"])
bread_BO = BelieveObject(names=["bread"])

with simulated_robot:   
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.33]).resolve().perform()

    pickup_pose_knife = CostmapLocation(target=bigknife_BO.resolve(), reachable_for=robot_desig).resolve()
    pickup_arm = pickup_pose_knife.reachable_arms[0]

    NavigateAction(target_locations=[pickup_pose_knife.pose]).resolve().perform()
    #PickUpAction(object_designator_description=bigknife_BO, arms=[left], grasps=["front"]).resolve().perform()
    PickUpAction(object_designator_description=bigknife_BO, 
                     arms=["left"], 
                     grasps=["top"]).resolve().perform()
    ParkArmsAction([Arms.BOTH]).resolve().perform()
with simulated_robot:   
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.33]).resolve().perform()
    
    cutting_standing_pose = CostmapLocation(target=bread_BO.resolve(), reachable_for=robot_desig).resolve()
    NavigateAction(target_locations=[cutting_standing_pose.pose]).resolve().perform()
 # MoveTCPMotion(target=self.pouring_location, arm=self.arm).resolve(). \
 #                perform()
 #            # sleep for some seconds
 #            time.sleep(self.wait_duration) # Sleep for wait_duration seconds
 #            MoveTCPMotion(target=self.revert_location, arm=self.arm).resolve(). \
 #                perform()
    LookAtAction(targets=[bread_BO.resolve().pose]).resolve().perform()
    
    detected_bread_desig = DetectAction(bread_BO).resolve().perform()
    
    
    

import numpy as np
import numpy as np

def calculate_and_transform_slices(bread_dimension, slice_thickness, bread_pose):
    """
    Calculate number of slices, transform them to the map frame, and provide coordinates with position and orientation.

    Args:
    - bread_dimension (tuple): Dimensions of the bread (length, width, height)
    - slice_thickness (float): Desired thickness of each slice in meters
    - bread_pose (Pose): Spawning pose of the bread with position and orientation

    Returns:
    - num_slices (int): Number of slices
    - coordinates_with_orientation (list): List of slice coordinates with position and orientation
    """
    
    # Calculate number of slices
    bread_length = bread_dimension[1]
    num_slices = int(bread_length // slice_thickness)
    
    # Calculate slice coordinates
    slice_coordinates = [i * slice_thickness for i in range(1, num_slices + 1)]
    
    # Transform slice coordinates to map frame with orientation
    coordinates_with_orientation = []
    for coordinate in slice_coordinates:
        x_in_map = bread_pose.position.x 
        y_in_map = bread_pose.position.y + coordinate
        z_in_map = bread_pose.position.z
        
        orientation = bread_pose.orientation
        coordinates_with_orientation.append({
            "position": [x_in_map, y_in_map, z_in_map],
            "orientation": orientation
        })
        
    return num_slices, coordinates_with_orientation



# Assuming bread_dim contains the dimensions of the bread (Placeholder dimensions for now, adjust as per actual value)
#bread_dim =  bread.get_Object_Dimensions()
bread_dim = (0.6, 0.2, 0.1)
# Given slice thickness is 3 cm or 0.03 meters
slice_thickness = 0.1
# Calculate slices and transform them to the map frame with orientation
num_slices, coordinates_with_orientation = calculate_and_transform_slices(bread_dim, slice_thickness, spawning_poses['bread'])
# Calculate slices and transform them to the map frame with orientation using the updated function




print(coordinates_with_orientation)

# Assuming you have an instance of the class that contains the add_vis_axis method

for coord in coordinates_with_orientation:
    # Convert the dictionary format to Pose object
    pose = Pose(coord["position"], coord["orientation"])

    world.add_vis_axis(pose)


## Adding Bread and Knife

Next, we'll define the spawning poses for our virtual bread and knife. We'll then add these objects to our virtual world.


In [ ]:
 bread_dim =  bread.get_Object_Dimensions()
print(bread_dim)


## Robot Actions

Now, we'll command our robot to perform certain actions, such as moving its torso or navigating to a specific position.


In [ ]:
world.remove_vis_axis()


## Bread Slicing Logic

The main goal of our simulation is to slice the virtual bread. To do this, we'll define a function, `calculate_and_transform_slices`, which calculates the positions and orientations of each slice based on the bread's dimensions, slice thickness, and the bread's pose in the world.


In [ ]:
world.remove_vis_axis()

In [ ]:
world.remove_vis_axis()